In [1]:
# pip install pymupdf4llm python-docx docx2pdf pandas

In [2]:
# pip install torch transformers langchain langchain_community


In [1]:
import os
directory = r'/home/ubuntu/AIML/Change_History'

In [4]:
import fitz  # PyMuPDF
import os

def remove_header_footer(input_pdf, output_pdf, header_height=130, footer_height=100):
    if not os.path.exists(input_pdf):
        raise FileNotFoundError(f"The file {input_pdf} does not exist.")

    # Open the PDF file
    doc = fitz.open(input_pdf)

    # Loop through the pages and remove header/footer based on coordinates
    for page_num in range(len(doc)):
        page = doc[page_num]

        # Define coordinates to remove (you can adjust based on your PDF layout)
        rect_header = fitz.Rect(0, 0, page.rect.width, header_height)  # Rectangle for header
        rect_footer = fitz.Rect(0, page.rect.height - footer_height, page.rect.width, page.rect.height)  # Rectangle for footer
        
        # Redact header and footer by drawing a white rectangle over them
        print(f"Redacting header on page {page_num + 1}")
        page.add_redact_annot(rect_header)
        
        print(f"Redacting footer on page {page_num + 1}")
        page.add_redact_annot(rect_footer)

        # Perform the actual redaction (make the redaction permanent)
        page.apply_redactions()

    # Save the modified PDF
    doc.save(output_pdf)
    doc.close()
    print(f"Redacted PDF saved as {output_pdf}")


# Usage
try:
    remove_header_footer(r'/home/ubuntu/AIML/Change_History/pdfs/SOP.EG.002 E1_637553751272743741.pdf', '2.pdf')
except Exception as e:
    print(e)


Redacting header on page 1
Redacting footer on page 1
Redacting header on page 2
Redacting footer on page 2
Redacting header on page 3
Redacting footer on page 3
Redacting header on page 4
Redacting footer on page 4
Redacting header on page 5
Redacting footer on page 5
Redacting header on page 6
Redacting footer on page 6
Redacted PDF saved as 2.pdf


In [5]:
import os

# Specify the directory containing the PDFs
directory = r'/home/ubuntu/AIML/Change_History/pdfs'

# Get a list of all PDF files in the directory
pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

# Assign names like doc1, doc2, etc.
named_pdfs = {f'doc{i+1}': file for i, file in enumerate(pdf_files)}

print(named_pdfs)

{'doc1': 'SOP.EG.002 E0_637520196028800408.pdf', 'doc2': '2.pdf', 'doc3': 'SOP-QA-028 E4_638588916127944936.pdf', 'doc4': 'SOP.EG.002 E1_637553751272743741.pdf', 'doc5': 'SOP-QA-028 E2_637482986350812499.pdf', 'doc6': 'SOP-QA-028 E3_637909810260553667.pdf', 'doc7': '1.pdf'}


In [6]:
named_pdfs

{'doc1': 'SOP.EG.002 E0_637520196028800408.pdf',
 'doc2': '2.pdf',
 'doc3': 'SOP-QA-028 E4_638588916127944936.pdf',
 'doc4': 'SOP.EG.002 E1_637553751272743741.pdf',
 'doc5': 'SOP-QA-028 E2_637482986350812499.pdf',
 'doc6': 'SOP-QA-028 E3_637909810260553667.pdf',
 'doc7': '1.pdf'}

In [7]:
keys = list(named_pdfs.keys())
doc1 = named_pdfs.get('doc7')
doc2 = named_pdfs.get('doc2')

# Output the result
print('Keys:', keys)
print('doc1:', doc1)
print('doc2:', doc2)

Keys: ['doc1', 'doc2', 'doc3', 'doc4', 'doc5', 'doc6', 'doc7']
doc1: 1.pdf
doc2: 2.pdf


In [8]:
import pymupdf4llm
import os
PATH = r"/home/ubuntu/AIML/Change_History/pdfs"
text1 = pymupdf4llm.to_markdown(os.path.join(PATH, doc1))
text2 = pymupdf4llm.to_markdown(os.path.join(PATH ,doc2))

Processing /home/ubuntu/AIML/Change_History/pdfs/1.pdf...
[                                        ] (0/5=======[========                                ] (1/5)

=======[================                        ] (2/5=======[========================                ] (3/5=======[================================        ] (4/5=======[========================================] (5/5]
Processing /home/ubuntu/AIML/Change_History/pdfs/2.pdf...
[                                        ] (0/6=====[======                                  ] (1/6======[=============                           ] (2/======[====================                    ] (3/6=====[==========================              ] (4/6======[=================================       ] (5/======[========================================] (6/6]


In [9]:
import re

def extract_sections(text):
    """
    Extracts sections from the provided text based on a pattern and returns a dictionary containing
    section numbers, titles, and their corresponding content.
 
    Args:
        text (str): The text to extract sections from.
 
    Returns:
        dict: A dictionary with section identifiers (e.g., **1.0 Introduction**) as keys and content as values.
    """
    section_pattern = r'(\*\*\d+\.\d+\*\*)\s+(\*\*.*?\*\*)\n\n(.*?)(?=\n\n\*\*\d+\.\d+\*\*|$)'
    sections_data = re.findall(section_pattern, text, re.DOTALL)
 
    section_data = {}
    for section_num, section_title, content in sections_data:
        section_id = f"{section_num} {section_title.strip()}"
        section_data[section_id] = content.strip()
 
    return section_data

# Example usage:
sections1 = extract_sections(text1)
sections2 = extract_sections(text2)

print(f"Sections for text 1 are: {sections1}")
print(f"Sections for text 2 are: {sections2}")

Sections for text 1 are: {'**2.0** **SCOPE**': 'This SOP is applicable for Breakdown Maintenance Handling for all equipments or systems\ninstalled in the General Block of Acme Formulation Pvt. Ltd. Nalagarh (H.P)', '**3.0** **RESPONSIBILITY**': '**3.1** Technician / Tech. Supervisor shall be responsible to attend the Breakdown Maintenance.', '**4.0** **ACCOUNTABILITY**': 'Department Head and QA Head shall be accountable for compliance of the SOP.', '**5.0** **PROCEDURE**': '**5.1** In the event of a breakdown, the concerned department will fill up the first section of the\nMaintenance Requisition Slip, as per format No. EG002/F01.', '**6.0** **ATTACHMENTS**': '**S.No.** **Title of Format** **Format Number**\n\n1 Maintenance Requisition Slip EG002/F01\n\n2 Quarterly Breakdown Analysis Report EG002/F02', '**7.0** **REFERENCES**': 'In – house.', '**8.0** **ABBREVIATIONS**': '|S.No.|Title of Format|Format Number|\n|---|---|---|\n|1|Maintenance Requisition Slip|EG002/F01|\n|2|Quarterly Brea

In [10]:
print(sections1)

{'**2.0** **SCOPE**': 'This SOP is applicable for Breakdown Maintenance Handling for all equipments or systems\ninstalled in the General Block of Acme Formulation Pvt. Ltd. Nalagarh (H.P)', '**3.0** **RESPONSIBILITY**': '**3.1** Technician / Tech. Supervisor shall be responsible to attend the Breakdown Maintenance.', '**4.0** **ACCOUNTABILITY**': 'Department Head and QA Head shall be accountable for compliance of the SOP.', '**5.0** **PROCEDURE**': '**5.1** In the event of a breakdown, the concerned department will fill up the first section of the\nMaintenance Requisition Slip, as per format No. EG002/F01.', '**6.0** **ATTACHMENTS**': '**S.No.** **Title of Format** **Format Number**\n\n1 Maintenance Requisition Slip EG002/F01\n\n2 Quarterly Breakdown Analysis Report EG002/F02', '**7.0** **REFERENCES**': 'In – house.', '**8.0** **ABBREVIATIONS**': '|S.No.|Title of Format|Format Number|\n|---|---|---|\n|1|Maintenance Requisition Slip|EG002/F01|\n|2|Quarterly Breakdown Analysis Report|EG0

In [11]:
import re

def normalize_sentence(sentence):
    """Normalize a sentence by stripping whitespace and reducing multiple spaces."""
    return ' '.join(sentence.strip().split())

def compare_sections(old_sections, new_sections):
    for section, old_content in old_sections.items():
        new_content = new_sections.get(section, '')

        # Print the section heading
        print(f"### {section}")
        
        # Changes Highlighted
        print("\n### Changes Highlighted: Sentence-wise Comparison")
        
        # Use regex to split by sentences while ignoring line breaks
        old_sentences = re.split(r'(?<=[.!?])\s+', old_content.replace('\n', ' '))
        new_sentences = re.split(r'(?<=[.!?])\s+', new_content.replace('\n', ' '))

        modified_sentences = []
        added_sentences = []
        removed_sentences = []

        # Convert lists of sentences into normalized forms
        normalized_old_sentences = [normalize_sentence(s) for s in old_sentences]
        normalized_new_sentences = [normalize_sentence(s) for s in new_sentences]

        # Compare each normalized sentence
        max_length = max(len(normalized_old_sentences), len(normalized_new_sentences))
        for i in range(max_length):
            # Get old and new sentences considering possible index out of range
            old_sentence = normalized_old_sentences[i] if i < len(normalized_old_sentences) else ''
            new_sentence = normalized_new_sentences[i] if i < len(normalized_new_sentences) else ''
            
            # Check if sentences differ in a meaningful way
            if old_sentence != new_sentence:
                if old_sentence or new_sentence:  # Ensure at least one is not empty
                    old_original = old_sentences[i].strip() if i < len(old_sentences) else ''
                    new_original = new_sentences[i].strip() if i < len(new_sentences) else ''
                    modified_sentences.append(f"- Old: '{old_original}' --> New: '{new_original}'")


        # Check for new sentences added
        added_sentences = new_sentences[len(old_sentences):]
        # Check for removed sentences
        removed_sentences = old_sentences[len(new_sentences):]

        # Print modified points
        if modified_sentences:
            print("1. Modified Points: ")
            for sentence in modified_sentences:
                print(f"    {sentence}")
        else:
            print("1. Modified Points: No modifications found.")

        # Print added points
        if added_sentences:
            print("2. Added Points:")
            for sentence in added_sentences:
                print(f"    - '{sentence.strip()}'")
        else:
            print("2. Added Points: No added content.")

        # Print removed points
        if removed_sentences:
            print("3. Removed Points:")
            for sentence in removed_sentences:
                print(f"    - '{sentence.strip()}'")
        else:
            print("3. Removed Points: No removed content.")

        # Provide a placeholder for contextual changes if needed
        print("\n### Contextual Changes")
        print("- Provide contextual analysis on the implications of the changes.")
        print("\n" + "=" * 50 + "\n")


compare_sections(sections1, sections2)


### **2.0** **SCOPE**

### Changes Highlighted: Sentence-wise Comparison
1. Modified Points: 
    - Old: 'This SOP is applicable for Breakdown Maintenance Handling for all equipments or systems installed in the General Block of Acme Formulation Pvt.' --> New: 'This SOP shall be applicable for Breakdown Maintenance Handling for all equipment’s or systems installed in General Block at Acme Formulation Pvt.'
    - Old: 'Nalagarh (H.P)' --> New: 'Nalagarh (H.P.).'
2. Added Points: No added content.
3. Removed Points: No removed content.

### Contextual Changes
- Provide contextual analysis on the implications of the changes.


### **3.0** **RESPONSIBILITY**

### Changes Highlighted: Sentence-wise Comparison
1. Modified Points: 
    - Old: '**3.1** Technician / Tech.' --> New: '**3.1** Technician shall be responsible to attend the Breakdown Maintenance.'
    - Old: 'Supervisor shall be responsible to attend the Breakdown Maintenance.' --> New: ''
2. Added Points: No added content.
3. Remove

JSON DICT

In [12]:
import json
import re

def normalize_sentence(sentence):
    """Normalize a sentence by stripping whitespace and reducing multiple spaces."""
    return ' '.join(sentence.strip().split())

def compare_sections(old_sections, new_sections):
    comparison_results = {}

    for section, old_content in old_sections.items():
        new_content = new_sections.get(section, '')

        section_result = {
            'section': section,
            'modified_points': [],
            'added_points': [],
            'removed_points': []
        }

        # Use regex to split by sentences while ignoring line breaks
        old_sentences = re.split(r'(?<=[.!?])\s+', old_content.replace('\n', ' '))
        new_sentences = re.split(r'(?<=[.!?])\s+', new_content.replace('\n', ' '))

        # Convert lists of sentences into normalized forms
        normalized_old_sentences = [normalize_sentence(s) for s in old_sentences]
        normalized_new_sentences = [normalize_sentence(s) for s in new_sentences]

        # Compare each normalized sentence
        max_length = max(len(normalized_old_sentences), len(normalized_new_sentences))
        for i in range(max_length):
            old_sentence = normalized_old_sentences[i] if i < len(normalized_old_sentences) else ''
            new_sentence = normalized_new_sentences[i] if i < len(normalized_new_sentences) else ''
            
            # Check if sentences differ in a meaningful way
            if old_sentence != new_sentence:
                if old_sentence or new_sentence:  # Ensure at least one is not empty
                    old_original = old_sentences[i].strip() if i < len(old_sentences) else ''
                    new_original = new_sentences[i].strip() if i < len(new_sentences) else ''
                    section_result['modified_points'].append({
                        'old': old_original,
                        'new': new_original
                    })

        # Check for new sentences added
        added_sentences = new_sentences[len(old_sentences):]
        section_result['added_points'] = [sentence.strip() for sentence in added_sentences]

        # Check for removed sentences
        removed_sentences = old_sentences[len(new_sentences):]
        section_result['removed_points'] = [sentence.strip() for sentence in removed_sentences]

        # Save comparison results for this section
        comparison_results[section] = section_result

    return comparison_results

differences_json = compare_sections(sections1, sections2)

json_file_path = 'comparison_results.json'

with open(json_file_path, 'w') as json_file:
    json.dump(differences_json, json_file, indent=4)

with open(json_file_path, 'r') as json_file:
    loaded_result = json.load(json_file)
    print(json.dumps(loaded_result, indent=4))


{
    "**2.0** **SCOPE**": {
        "section": "**2.0** **SCOPE**",
        "modified_points": [
            {
                "old": "This SOP is applicable for Breakdown Maintenance Handling for all equipments or systems installed in the General Block of Acme Formulation Pvt.",
                "new": "This SOP shall be applicable for Breakdown Maintenance Handling for all equipment\u2019s or systems installed in General Block at Acme Formulation Pvt."
            },
            {
                "old": "Nalagarh (H.P)",
                "new": "Nalagarh (H.P.)."
            }
        ],
        "added_points": [],
        "removed_points": []
    },
    "**3.0** **RESPONSIBILITY**": {
        "section": "**3.0** **RESPONSIBILITY**",
        "modified_points": [
            {
                "old": "**3.1** Technician / Tech.",
                "new": "**3.1** Technician shall be responsible to attend the Breakdown Maintenance."
            },
            {
                "old": "Supe

In [13]:
import json

# Lists to hold all modified, added, and removed points
modified_points_list = []
added_points_list = []
removed_points_list = []

for section, content in differences_json.items():
    if section == "**9.0** **REVISION HISTORY**":
        break 
    modified_points_list.extend(content["modified_points"])  # Add all modified points
    added_points_list.extend(content["added_points"])        # Add all added points
    removed_points_list.extend(content["removed_points"])    # Add all removed points

modified_points_list.sort(key=lambda x: x['new'])

print("Modified Points:")
for point in modified_points_list:
    print(f"{point['new']}\n")

print("\nAdded Points:")
for point in added_points_list:
    print(point)

print("\nRemoved Points:")
for point in removed_points_list:
    print(point)


Modified Points:


**3.1** Technician shall be responsible to attend the Breakdown Maintenance.

**Abbreviation** **Description**  EDCS Electronic Document Control System  EG Engineering  MR Maintenance Requisition  QA Quality Assurance  SOP Standard Operating Procedure

**S.

-----  5.13 After evaluation of QA tick in the acceptance note that the equipment is available for routine use by sign/ date of requesting department, checked by engineering and QA department.

5.1 In the event of a breakdown, concerned department shall fill up the first section of the Maintenance Requisition Slip, as per format no.

5.10 Engineering personnel shall perform the repair work.

5.11 After repairs of machine & other work, the engineering personnel shall fill in the white copy, the third section (to be filled by engineering department), indicating the summary of repair work done.

5.14 Engineering personnel shall retain the white copy of work request.

5.15 The white copy of work request shall be atta

In [ ]:
import os
import fitz  # PyMuPDF

# Directory where highlighted PDFs will be saved


# Example Usage
if __name__ == "__main__":
    new_pdf_path = r"/home/ubuntu/AIML/Change_History/pdfs/SOP.EG.002 E1_637553751272743741.pdf"  # Path to the new PDF
    old_pdf_path = r"/home/ubuntu/AIML/Change_History/pdfs/SOP.EG.002 E0_637520196028800408.pdf"  # Path to the old PDF

    # Replace these with your actual lists of points
    modified_points_list = [{'new': 'example modified text'}]  # Example of modified points
    added_points_list = [{'text': 'example added text'}]  # Example of added points
    removed_points_list = [{'text': 'example removed text'}]  # Example of removed points

    # Highlight points in the new version
    highlighted_new_pdf = highlight_new_version_pdf(new_pdf_path, modified_points_list, added_points_list)
    print(f"Highlighted PDF saved at: {highlighted_new_pdf}")

    # Highlight points in the old version
    highlighted_old_pdf = highlight_old_version_pdf(old_pdf_path, removed_points_list)
    print(f"Highlighted PDF saved at: {highlighted_old_pdf}")


Highlighted PDF saved at: highlighted_pdfs/SOP.EG.002 E1_637553751272743741_highlighted_new.pdf
Highlighted PDF saved at: highlighted_pdfs/SOP.EG.002 E0_637520196028800408_highlighted_old.pdf
